In [29]:
import pandas as pd
import numpy as np
import os
import sys

fileDir = os.path.dirname(os.path.realpath('__file__'))
print(fileDir)
file = "../../../Received_data_for_tests.csv"

/Users/mathiasmarkvardsen/bachelor/IOTunderwater/Code/dsp/plotting & data handling


In [30]:
df = pd.read_csv(file, sep=',')
df.tail(2)

,ID,Bitrate,Carrier Frequency,Original Message,Decoded without bandpass,Hamming Dist without bandpass,Decoded with bandpass,Hamming Dist with bandpass,Encoding,Transmitter,Container,Speaker depth,Distance to speaker,Test description,Original message in bits,Data bits without bandpass,Data bits with bandpass
15,2511c487-c2e7-4ec3-b655-b15b5aadcf32,100,6000,Hello_there,Hello_there,0.0,Hello_t`%re,2.0,No Encoding,ESP,plastic,200,600,EXAMPLE 2: test 2,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,..."
16,1b7c3894-23bf-441c-b4cb-b1686caa48e4,100,6000,Hello_there,No preamble found,NaN,No preamble found,NaN,No Encoding,ESP,plastic,200,600,EXAMPLE 2: test 2,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...",NaN,NaN


#### Comparison of original and decoded bits if the len of decoded bits is identical to the original bits

In [37]:
# Grouping by 'test description'
grouped = df.groupby('Test description')
grouped.head()



,ID,Bitrate,Carrier Frequency,Original Message,Decoded without bandpass,Hamming Dist without bandpass,Decoded with bandpass,Hamming Dist with bandpass,Encoding,Transmitter,Container,Speaker depth,Distance to speaker,Test description,Original message in bits,Data bits without bandpass,Data bits with bandpass
0,267822ee-72ae-4613-bc3a-05fcf3ed53c3,100,6000,Hello_there,Hello_there,0.0,Hello_there,0.0,No Encoding,ESP,plastic,200,600,EXAMPLE : This test aims to create a function ...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,..."
1,d0f80cfc-c53f-43bc-825a-90b3ef356358,100,6000,Hello_there,Hello_there,0.0,Hello_there,0.0,No Encoding,ESP,plastic,200,600,EXAMPLE : This test aims to create a function ...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,..."
2,e5651823-e371-46ca-a308-0de984160d94,100,6000,Hello_there,Hello_there,0.0,Hello_there,0.0,No Encoding,ESP,plastic,200,600,EXAMPLE : This test aims to create a function ...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,..."
3,96b5bd19-51d2-4ac2-a29e-e6c6a8ee3c1f,100,6000,Hello_there,Hello_there,0.0,Hello_there,0.0,No Encoding,ESP,plastic,200,600,EXAMPLE : This test aims to create a function ...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,..."
4,e1dff7fb-97f0-4d13-a3c6-74b4bad8cc07,100,6000,Hello_there,Hello_there,0.0,Hello_there,0.0,No Encoding,ESP,plastic,200,600,EXAMPLE : This test aims to create a function ...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,..."
7,e2dd0a56-f36b-4a92-aab8-6bcad932b9c6,100,6000,Hello_there,Hello_there,0.0,Hello_there,0.0,No Encoding,ESP,plastic,200,600,EXAMPLE 2: test 2,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,..."
8,91c9e55d-c1e1-46dd-b805-7abc5968c4a7,100,6000,Hello_there,Hello_there,0.0,Hello_there,0.0,No Encoding,ESP,plastic,200,600,EXAMPLE 2: test 2,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,..."
9,3a327a40-47fd-4196-a377-3a3493d7af28,100,6000,Hello_there,Hello_there,0.0,Hello_there,0.0,No Encoding,ESP,plastic,200,600,EXAMPLE 2: test 2,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,..."
10,dcfab54b-2323-42fa-ba56-5837f160310b,100,6000,Hello_there,Hello_there,0.0,Hello_there,0.0,No Encoding,ESP,plastic,200,600,EXAMPLE 2: test 2,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,..."
11,3d25ad41-281a-430a-adf2-abb482af76f6,100,6000,Hello_there,Hello_there,0.0,Hello_there,0.0,No Encoding,ESP,plastic,200,600,EXAMPLE 2: test 2,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,...","[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,..."


In [38]:
import ast
import numpy as np

# Define the Barker 13 sequence in binary form
barker_13 = [1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1]

def parse_bit_list(bit_string):
    try:
        return ast.literal_eval(bit_string)
    except:
        return []

def check_barker_similarity(decoded_bits, barker_seq=barker_13, threshold=11):
    """Check if any subsequence in decoded bits matches the Barker sequence with at least `threshold` matching bits"""
    decoded_bits = np.array(decoded_bits)
    barker_len = len(barker_seq)
    for i in range(len(decoded_bits) - barker_len + 1):
        subseq = decoded_bits[i:i + barker_len]
        match_count = np.sum(subseq == barker_seq)
        if match_count >= threshold:
            return True, i
    return False, -1

def analyze_row(row):
    orig_bits = parse_bit_list(row['Original message in bits'])
    decoded_bits = parse_bit_list(row['Data bits without bandpass'])

    if isinstance(decoded_bits[0], list):  # Some entries are double-nested
        decoded_bits = decoded_bits[0]

    result = {
        'ID': row['ID'],
        'Same Length': len(orig_bits) == len(decoded_bits),
        'Length Original': len(orig_bits),
        'Length Decoded': len(decoded_bits),
    }

    if len(orig_bits) != len(decoded_bits):
        similar, pos = check_barker_similarity(decoded_bits)
        result['Barker-like sequence found'] = similar
        result['Barker match start index'] = pos if similar else None
        result['0->1'] = None
        result['1->0'] = None
    else:
        orig_bits = np.array(orig_bits)
        decoded_bits = np.array(decoded_bits)
        result['0->1'] = int(np.sum((orig_bits == 0) & (decoded_bits == 1)))
        result['1->0'] = int(np.sum((orig_bits == 1) & (decoded_bits == 0)))
        result['Barker-like sequence found'] = None
        result['Barker match start index'] = None

    return result

# Apply analysis
analysis_results = df.apply(analyze_row, axis=1)
analysis_df = pd.DataFrame(list(analysis_results))

analysis_df.head()

def analyze_row_safe(row):
    try:
        orig_bits = parse_bit_list(row['Original message in bits'])
        decoded_bits = parse_bit_list(row['Data bits without bandpass'])

        if not orig_bits or not decoded_bits:
            return {
                'ID': row['ID'],
                'Same Length': False,
                'Length Original': len(orig_bits),
                'Length Decoded': len(decoded_bits),
                '0->1': None,
                '1->0': None,
                'Barker-like sequence found': None,
                'Barker match start index': None,
                'Error': 'Empty or malformed bit list'
            }

        if isinstance(decoded_bits[0], list):  # Handle nested lists
            decoded_bits = decoded_bits[0]

        result = {
            'ID': row['ID'],
            'Same Length': len(orig_bits) == len(decoded_bits),
            'Length Original': len(orig_bits),
            'Length Decoded': len(decoded_bits),
            'Error': None
        }

        if len(orig_bits) != len(decoded_bits):
            similar, pos = check_barker_similarity(decoded_bits)
            result['Barker-like sequence found'] = similar
            result['Barker match start index'] = pos if similar else None
            result['0->1'] = None
            result['1->0'] = None
        else:
            orig_bits = np.array(orig_bits)
            decoded_bits = np.array(decoded_bits)
            result['0->1'] = int(np.sum((orig_bits == 0) & (decoded_bits == 1)))
            result['1->0'] = int(np.sum((orig_bits == 1) & (decoded_bits == 0)))
            result['Barker-like sequence found'] = None
            result['Barker match start index'] = None

        return result
    except Exception as e:
        return {
            'ID': row.get('ID', 'unknown'),
            'Same Length': False,
            'Length Original': 0,
            'Length Decoded': 0,
            '0->1': None,
            '1->0': None,
            'Barker-like sequence found': None,
            'Barker match start index': None,
            'Error': str(e)
        }

# Apply the safe version of the analysis
analysis_df = df.apply(analyze_row_safe, axis=1, result_type='expand')
analysis_df.head()


IndexError: list index out of range